In [13]:
import pandas as pd
from decimal import Decimal

from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-3

Initialize a fast simulator

It is the result of trading on a single order, which

sells 1B USDC at a price of 1500 USDC per ETH (price range is from 1500 to 1000, but the start price, which each one of the trades above goes by, is 1500)

In [14]:
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

# FastSim.debug = True

In [15]:
FastSim.add_strategy('USDC', Decimal(1000000000), Decimal(1500), Decimal(1000), 0, None, None)
FastSim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,1.000000e+09,1.000000e+09,USDC,False,1500.0,1000.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,ETH,0.000000e+00,0.000000e+00,ETH,True,None,None,None,USDC per ETH,0


## Route by Source

#### AMM buys ETH

* tradeBySourceAmount: input =     10.0 ETH , output =   14999.958711 USDC, rate = 1499.9958711
* tradeBySourceAmount: input =    100.0 ETH , output =  149995.871286 USDC, rate = 1499.95871286
* tradeBySourceAmount: input =   1000.0 ETH , output = 1499587.230921 USDC, rate = 1499.587230921

In [16]:
FastSim.trader_sells('ETH',Decimal(10), execute=False)
FastSim.trader_sells('ETH',Decimal(100), execute=False)
FastSim.trader_sells('ETH',Decimal(1000), execute=False)
results = FastSim.state()['trades']

# we can see that the output is as expected below
results

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,None,1.499996e+04,USDC,10.0,ETH,ETHUSDC,0,1,1499.995871,USDC per ETH
0,0,0,A,AMM sells 15000USDC buys 10ETH,True,False,None,1.499996e+04,USDC,10.0,ETH,ETHUSDC,[0],1,1499.995871,USDC per ETH
0,1.0,1,0,route #0,False,False,None,1.499959e+05,USDC,100.0,ETH,ETHUSDC,0,1,1499.958713,USDC per ETH
0,1,1,A,AMM sells 149996USDC buys 100ETH,True,False,None,1.499959e+05,USDC,100.0,ETH,ETHUSDC,[0],1,1499.958713,USDC per ETH
0,2.0,2,0,route #0,False,False,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,0,1,1499.587231,USDC per ETH
0,2,2,A,AMM sells 1499587USDC buys 1000ETH,True,False,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,[0],1,1499.587231,USDC per ETH


Now, verify that changing current_rate works or goes in the opposite direction

In [17]:
# Re-initialize to origin state for clarity / readability of output
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)
FastSim.add_strategy('USDC', Decimal(1000000000), Decimal(1500), Decimal(1000), 0, None, None)

# Now, verify that changing current_rate works or goes in the opposite direction
# we do this by setting execute=True which will change the current_rate accordingly
FastSim.trader_sells('ETH',Decimal(1000), execute=True)
FastSim.trader_sells('ETH',Decimal(1000), execute=True)
FastSim.trader_sells('ETH',Decimal(1000), execute=True)

# the trader sells ETH, increasing the supply - therefore we would expect the current_rate / price to go down, and it does
results = FastSim.state()['trades']
results

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,0,1,1499.587231,USDC per ETH
0,0,0,A,AMM sells 1499587USDC buys 1000ETH,True,True,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,[0],1,1499.587231,USDC per ETH
0,1.0,1,0,route #0,False,True,None,1.498762e+06,USDC,1000.0,ETH,ETHUSDC,0,1,1498.762147,USDC per ETH
0,1,1,A,AMM sells 1498762USDC buys 1000ETH,True,True,None,1.498762e+06,USDC,1000.0,ETH,ETHUSDC,[0],1,1498.762147,USDC per ETH
0,2.0,2,0,route #0,False,True,None,1.497938e+06,USDC,1000.0,ETH,ETHUSDC,0,1,1497.937744,USDC per ETH
0,2,2,A,AMM sells 1497938USDC buys 1000ETH,True,True,None,1.497938e+06,USDC,1000.0,ETH,ETHUSDC,[0],1,1497.937744,USDC per ETH


## Route by Target

#### Trader Buys USDC

* tradeByTargetAmount: input =   1500.0 USDC, output =       1.000001 ETH , rate =    0.000666667
* tradeByTargetAmount: input =  15000.0 USDC, output =      10.000028 ETH , rate =    0.000666668
* tradeByTargetAmount: input = 150000.0 USDC, output =     100.002753 ETH , rate =    0.000666685


In [18]:
# Re-initialize to origin state for clarity / readability of output
FastSim = CarbonSimulatorUI(pair="USDC/ETH", verbose=False, matching_method='fast', raiseonerror=True)
FastSim.add_strategy('USDC', Decimal(1000000000), 1/Decimal(1500), 1/Decimal(1000))
, 0, None, None)

# Now perform route by target trade
FastSim.trader_buys('USDC',Decimal(1500), execute=False)
FastSim.trader_buys('USDC',Decimal(15000), execute=False)
FastSim.trader_buys('USDC',Decimal(150000), execute=False)
results = FastSim.state()['trades']

# Note that the results are not quite as expected (per Barak's output in bullets above, where price=0.00673 matches but the others do not)
results

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,None,2.000000,USDC,1500.0,ETH,USDCETH,0,1,0.00133333351644446953361589571374371792748765...,ETH per USDC
0,0,0,A,AMM sells 2USDC buys 1500ETH,True,False,None,2.000000,USDC,1500.0,ETH,USDCETH,[0],1,0.00133333351644446959170715726111626385996690...,ETH per USDC
0,1.0,1,0,route #0,False,False,None,11.000028,USDC,15000.0,ETH,USDCETH,0,1,0.00073333516857792619051699417285796870857927...,ETH per USDC
0,1,1,A,AMM sells 11USDC buys 15000ETH,True,False,None,11.000028,USDC,15000.0,ETH,USDCETH,[0],1,0.00073333516857792621645148262481375019222456...,ETH per USDC
0,2.0,2,0,route #0,False,False,None,101.002753,USDC,150000.0,ETH,USDCETH,0,1,0.00067335168423167377803658208142496572386554...,ETH per USDC
0,2,2,A,AMM sells 101USDC buys 150000ETH,True,False,None,101.002753,USDC,150000.0,ETH,USDCETH,[0],1,0.00067335168423167381142182759982903331396684...,ETH per USDC
